In [1]:
import sys
import os
import shutil
sys.path.append('C://Repos/AHAKLWE')
from extract import extract 
import pandas as pd

In [19]:
file_path = './IMA 2023 ACR signing version.pdf'
df = extract(file_path, tag_tables = True) 
#mask = (df.tag != "p")
#df['label'] = None
#df.loc[mask,'label']= 0
# df.to_csv('advocard_main.csv')

page dims for camelot:  (595.3200073242188, 841.9199829101562)
------ Function extract_table  Took 0.6833 seconds ----------
------ Function extract  Took 1.0397 seconds ----------


In [29]:
import fitz
doc = fitz.open(file_path)

In [55]:
print(doc[25].get_text().encode('utf-8').decode())


�
�
�
�
�
�
�
��O����������F�G�����������������������������
)#��'����������������;��� 8��#��(��4��8����������/�������������������������������� ����� ���"��E��#�4���������������4�����
 8���������������������?����#��������#��7��/�����?����������#����#�����������������������������������4��������=�������#��
������4�������4����������
�
6�������3�2�������&�����01��������������������������  �������������������
����������
&�����0�
&�����0��  �����
&���������
&�����0�
�0���
&���� ���� ��������  ������
��
��
��
��
��
��
.U'9:0(',9�
'9:*B�9���'':�
����������9�
*B���E(��E:�52�
��
9E����������������$$$��
.U'9:0(',9�
'9:*B�9���'':�
����������E�
*B���E(��E:�52�
��
9E����������������$$$��
.U'9:0(',9�
'9:E.&9���'':�
E.&9���'':�
),E�0,�
��
9E����������������$$$��
.U'9:0(',9�
'9:0(A',E��
'9:0(A',E��
',E��&3*�(,�3Q�
.��CE,�
9E����������������$$$��
.U'9:0(',9�
'9:0(AN)(&,�
'9:0(AN)(&,�
N)(�&,0C��
.��CE,�
9E����������������$$$��
�
����������8)�"�&������� �����C������*�.����������F9AG��-�+P�-**�
�
�
6�������

# OCR on pdf. 

Install tesseract engine (CLI by google). For windows: download from: https://github.com/UB-Mannheim/tesseract/wiki and run
For Ubuntu (on the VM later): sudo apt-get install tesseract-ocr

Selected Additional Language support for danish and German - might need more 

Add to Path (environment variables)
Restart PC

pip install ocrmypdf
pip install pytesseract
pip install opencv-python
pip install pdf2image

C:\Users\KLWE\AppData\Local\Programs\Tesseract-OCR

In [46]:
# import libs
try:
    from PIL import Image
except ImportError:
    import Image
import cv2
import pytesseract
import os
import numpy as np
import pandas as pd
import re
from pdf2image import convert_from_bytes

In [47]:
file_path = './IMA 2023 ACR signing version.pdf'

In [77]:

# Some help functions 
def get_conf(page_gray):
    '''return a average confidence value of OCR result '''
    df = pytesseract.image_to_data(page_gray,output_type='data.frame')
    df.drop(df[df.conf==-1].index.values,inplace=True)
    df.reset_index()
    return df.conf.mean()
  
def deskew(image):
    '''deskew the image'''
    gray = cv2.bitwise_not(image)
    temp_arr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(temp_arr > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated
  
'''
Main part of OCR:
pages_df: save eextracted text for each pdf file, index by page
OCR_dic : dict for saving df of each pdf, filename is the key
'''




# convert pdf into image
pdf_file = convert_from_bytes(open(file_path, 'rb').read())
# create a df to save each pdf's text
texts = []
PAGE_NUM = 30
for (i,page) in enumerate(pdf_file) :
    if i == PAGE_NUM:
        try:
            # transfer image of pdf_file into array
            page_arr = np.asarray(page)
            # transfer into grayscale
            page_arr_gray = cv2.cvtColor(page_arr,cv2.COLOR_BGR2GRAY)
            # deskew the page
            page_deskew = deskew(page_arr_gray)
            # cal confidence value
            page_conf = get_conf(page_deskew)
            # extract string 
            texts.append(pytesseract.image_to_data(page_deskew, lang = "eng+ger", output_type=pytesseract.Output.DATAFRAME))
            # pages_df = pd.concat([pages_df, pd.DataFrame({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)})]) #(pages_df.append({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)}, ignore_index=True)
        except:
            # if can't extract then give some notes into df
            texts.append('N/A')
            #¤pages_df = pd.concat([pages_df, pd.DataFrame({'conf': -1 ,'text':'N/A'})]) #pages_df.append({'conf': -1,'text': 'N/A'}, ignore_index=True)
            continue
    # save df into a dict with filename as key        

Output format: 

* level: hierarchical layout (a word is in a line, which is in a paragraph, which is in a block, which is in a page), a value from 1 to 5
1: page
2: block
3: paragraph
4: line
5: word
* page_num: when provided with a list of images, indicates the number of the file, when provided with a multi-pages document, indicates the page number, starting from 1
* block_num: block number within the page, starting from 0
* par_num: paragraph number within the block, starting from 0
* line_num: line number within the paragraph, starting from 0
* word_num: word number within the line, starting from 0
* left: x coordinate in pixels of the text bounding box top left corner, starting from the left of the image
* top: y coordinate in pixels of the text bounding box top left corner, starting from the top of the image
* width: width of the text bounding box in pixels
* height: height of the text bounding box in pixels
* conf: confidence value, from 0 (no confidence) to 100 (maximum confidence), -1 for all level except 5
* text: detected text, empty for all levels except 5

In [78]:
df = texts[0]

In [79]:
df.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text'],
      dtype='object')

In [80]:
line_text = df[df.level == 5].groupby(['block_num','line_num']).agg({'text': lambda x:' '.join(x)}).reset_index()
line_pos = df[df.level == 4][['block_num', 'line_num', 'left','top', 'width','height']]

In [81]:
df = pd.merge(line_pos, line_text,  how='left', on=['block_num','line_num'])

In [82]:
import fitz
doc = fitz.open(file_path)
PAGE_WIDTH, PAGE_HEIGHT = doc[29].rect.width, doc[29].rect.height
IMG_HEIGHT, IMG_WIDTH = page_deskew.shape

In [106]:
df = df.assign(
    ymin = df.top / IMG_WIDTH * PAGE_WIDTH,
    ymax = (df.top+ df.height) / IMG_WIDTH * PAGE_WIDTH,
    xmax = (((df.left + df.width) / IMG_HEIGHT)) * PAGE_HEIGHT,
    xmin = (((df.left) / IMG_HEIGHT)) * PAGE_HEIGHT
)
df = df.assign(
    is_upper = df.text.apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).isupper()),
    is_bold = 'N/A',
    span_font = 'N/A',
    font_size = df.height,
    page_num = PAGE_NUM, 
    tag = None, 
    label = None
)[['xmin','ymin','xmax','ymax','text','is_upper','is_bold','span_font','font_size','page_num','block_num','tag','label']]


In [108]:
def add_tag(x, pr_5, pr_95, font_dict, page_height, page_width):
    if x.font_size > pr_95: 
        tag = 'h'
    elif (x.font_size < pr_5) \
            or (re.search(r'\bpage\s+\d*.*\d\b', x.text.lower())) \
            or ((x.ymin > 0.95*page_height) and (x.xmin > 0.7*page_width)) \
            or ((x.ymin > 0.95*page_height) and (x.xmin > 0.7*page_width)): 
        tag = 'f'
    elif (font_dict[x.span_font] < 0.01):  # speacial font - occurs only in one % 
        tag = 's'
    else: 
        tag = 'p'

    return(tag)

In [109]:
pr_5 = np.percentile(df.font_size, 5)
pr_95 = np.percentile(df.font_size, 95)
font_dict = (df.groupby('span_font').count()['text']/len(df)).to_dict()
df['tag'] = df.apply(
        lambda x: add_tag(x, pr_5, pr_95, font_dict, page_height=PAGE_HEIGHT, page_width=PAGE_WIDTH) , axis = 1
    )

In [86]:
from fitz import Rect

def coords_to_rect(xmin, ymin, xmax, ymax):
    quad = (xmin, ymin, xmax, ymax)
    return Rect(quad)

def highlight_area(rect: Rect, page_no: int, pdf_path: str, *args, **kwargs):
    doc = fitz.open(pdf_path)
    doc[page_no].add_highlight_annot(rect)
    if kwargs['output_file']:
        doc.save(kwargs['output_file'])
    else: 
        doc.save(pdf_path)
    doc.close()   

In [99]:
highlight_area(coords_to_rect(df.iloc[3].xmin, df.iloc[3].ymin, df.iloc[3].xmax, df.iloc[3].ymax), page_no = 30, pdf_path=file_path, output_file = 'test.pdf')

In [1]:
import pandas as pd
df = pd.read_csv('wien_main_update.csv', index_col = 0)
df.assign(label = None).to_csv('wien_main_update.csv')